# Web Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import etree

verbose = True


## Elo Ratings

&#9632; For Fifa Ranking by year <br>
&#9632; Has data on Matches [Data can be grabbed from results.csv] <br> 
<ol>
    &#9633; Home/Away/Neutral <br>
    &#9633; Wins/Losses/Draws <br>
    </ol>

In [67]:
URL = 'https://eloratings.net/'
headers = {"User-Agent":"Mozilla/5.0"}
requested = requests.get(URL) # get URL
verbose = True

if verbose: # print raw html content
    print('Raw HTML content \n', requested.content, '\n\n')

soup = BeautifulSoup(requested.content, 'lxml')

if verbose:
    print('Prettified Soup \n', soup.prettify(), '\n\n')

Raw HTML content 
 b'<!DOCTYPE html>\n<html lang="en"><head><title>World Football Elo Ratings</title>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n<meta name="description" content="Ratings for national football teams based on the Elo rating system.">\n<meta name="keywords" content="football, ratings, Elo, rankings, national, international, soccer, teams">\n<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no">\n<link rel="stylesheet" href="scripts/slick.grid.css" type="text/css"/>\n<link rel="stylesheet" href="scripts/dygraph.css" type="text/css"/>\n<script type="text/javascript" src="scripts/dygraph.js"></script>\n<script type="text/javascript" src="scripts/jquery.js"></script>\n<script type="text/javascript" src="scripts/slick.core.js"></script>\n<script type="text/javascript" src="scripts/slick.grid.js"></script>\n<script type="text/javascript" src="scripts/cldr.js"></script>\n<script t


## Transfer Market

&#9632; Squad Size <br>
&#9632; Average Age <br> 
&#9632; Total Value <br>
&#9632; Points <br>
&#9632; Confederation <br>
<ol>
    &#9633; Valuable b/c could get confederation data to use <br>
    &#9633; Potential Issue Confederation stats only, CONACAF not as good as UEFA <br>
    </ol>

In [57]:
def get_Fifa_Rankings(URL, rankings, headers): # function for grabbing fifa rankings from one page
    requested = requests.get(URL, headers = headers)
    soup = BeautifulSoup(requested.content, 'lxml')
    dom = etree.HTML(str(soup)) #. beautiful soup stuff
    
    base_path = '//*[@id="yw1"]/table/tbody/tr['
    xpath = '1]/td[2]/a[2]' # xpath addition
    # xpath = '//*[@id="yw1"]/table/tbody/tr[1]/td[2]/a[2]' for team name
    # xpath = '//*[@id="yw1"]/table/tbody/tr[1]/td[i]' for confederation(i = 3) for ranking(i=4)
    
    teams = 25
    currentRow = ['Name', 'Fed', 'Rank'] # blank template
    for i in range(teams):
        xpath = base_path + str(i+1) + ']/td['
        teamPath = xpath + '2]/a[2]'
        currentRow[0] = dom.xpath(teamPath)[0].text
        for j in range(1,3):
            path = xpath + str(j+2) + ']'
            currentRow[j] = dom.xpath(path)[0].text
        print(currentRow)
        currentRow = ['Name', 'Fed', 'Rank']
        
    #print(dom.xpath(xpath)[0].text)
    return
    

In [65]:
baseURL = 'https://www.transfermarkt.us/statistik/weltrangliste/statistik/stat/ajax/yw1/datum/' # URL of website w/ chart or data
headers = {"User-Agent":"Mozilla/5.0"}
rankings = []
pages = 3

for i in range(pages):
    date = [2022, '02', 10] # date most recetn
    newDate = ''
    for x in range(len(date)):
        if x ==2:
            newDate += str(date[x])
        else:
            newDate += str(date[x])+'-'
    
    
    dateURL = newDate + '/plus/0/galerie/0/page/'
    get_Fifa_Rankings((baseURL + dateURL + str(i+1)), rankings, headers)


['Belgium', 'UEFA', '1828']
['Brazil', 'CONMEBOL', '1823']
['France', 'UEFA', '1786']
['Argentina', 'CONMEBOL', '1767']
['England', 'UEFA', '1756']
['Italy', 'UEFA', '1741']
['Spain', 'UEFA', '1705']
['Portugal', 'UEFA', '1660']
['Denmark', 'UEFA', '1655']
['Netherlands', 'UEFA', '1654']
['Germany', 'UEFA', '1648']
['Mexico', 'CONCACAF', '1648']
['USA', 'CONCACAF', '1643']
['Switzerland', 'UEFA', '1643']
['Croatia', 'UEFA', '1621']
['Uruguay', 'CONMEBOL', '1614']
['Sweden', 'UEFA', '1588']
['Senegal', 'CAF', '1588']
['Colombia', 'CONMEBOL', '1586']
['Wales', 'UEFA', '1578']
['Iran', 'AFC', '1573']
['Peru', 'CONMEBOL', '1563']
['Japan', 'AFC', '1550']
['Morocco', 'CAF', '1547']
['Serbia', 'UEFA', '1547']
['Chile', 'CONMEBOL', '1543']
['Ukraine', 'UEFA', '1535']
['Poland', 'UEFA', '1531']
['South Korea', 'AFC', '1523']
['Austria', 'UEFA', '1512']
['Czechia', 'UEFA', '1510']
['Nigeria', 'CAF', '1510']
['Canada', 'CONCACAF', '1498']
['Egypt', 'CAF', '1497']
['Russia', 'UEFA', '1493']
['Tun


## Fifa Index

&#9632; Get Fifa video game team rankings <br>
<ol>
    &#9633; ATT <br>
    &#9633; MID <br>
    &#9633; DEF <br>
    &#9633; OVR <br>
    &#9633; Team Rating <br>
    </ol>
    
&#9632; How much of this is UNIQUE data? <br>
&#9632; Easier to calculate OVR? <br>

In [15]:
def get_Fifa_Index(URL, teamRank):
    webpage = requests.get(URL)
    soup = BeautifulSoup(webpage.content, 'lxml')
    dom = etree.HTML(str(soup))
    # /a for team name and /span for int
    # tr[i]/td[j], where (i-2) is team rank, where j is column {2; Team Name, 4; ATT, 5; MID, 6; DEF, 7; OVR}  

    moreTeams = True
    currentRow = ['Name', 'ATT', 'MID', 'DEF', 'Date']
    i=0
    while moreTeams:

        teamVal = i+3
        xpath = '/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[' + str(teamVal) + ']/td[2]/a'
        try:
            #print('try:', i)
            currentRow[0]=(dom.xpath(xpath)[0].text)
            for j in range(4, 7):
                xpath = '/html/body/main/div/div/div[2]/div[2]/table/tbody/tr[' + str(teamVal) + ']/td[' + str(j) + ']/span'
                #print(xpath)
                currentRow[j-3]=(dom.xpath(xpath)[0].text)
            currentRow[4] = dom.xpath('/html/body/main/div/div/div[2]/nav[1]/ol/li[3]/a')[0].text
            teamRank.append(currentRow)
            currentRow = ['Name', 'ATT', 'MID', 'DEF', 'Date']
            count = 0
            i +=1
        except:
            #print('except: ', i)
            count += 1
            i += 1
            if count == 5:
                break
            pass
    return teamRank

In [16]:
base_url = 'https://www.fifaindex.com/teams/fifa'
# URL = 'https://www.fifaindex.com/teams/fifa22/?page=1&type=1'
teamRank =[]
year = 22
page = 1

while year > 4:
    if year < 10:
        year = '0'+str(year)
    else:
        year = str(year)
    URL = base_url + year + '/?page=' + str(page) + '&type=1'
    teamRank = get_Fifa_Index(URL, teamRank)
    year = int(year)
    if page == 1:
        page += 1
    else:
        page = 1
        year -= 1
    
for x in range(len(teamRank)):
    print('%i, %s, %s, %s, %s, %s' % (x, teamRank[x][0], teamRank[x][1], teamRank[x][2], teamRank[x][3], teamRank[x][4]))

0, France, 85, 86, 83, April 25, 2022 
1, Germany, 83, 85, 80, April 25, 2022 
2, Portugal, 85, 83, 85, April 25, 2022 
3, Spain, 83, 85, 83, April 25, 2022 
4, England, 86, 83, 83, April 25, 2022 
5, Argentina, 86, 81, 81, April 25, 2022 
6, Italy, 84, 84, 82, April 25, 2022 
7, Belgium, 83, 82, 80, April 25, 2022 
8, Holland, 81, 82, 83, April 25, 2022 
9, Brazil, 80, 80, 80, April 25, 2022 
10, Denmark, 77, 78, 79, April 25, 2022 
11, Poland, 81, 74, 75, April 25, 2022 
12, Mexico, 80, 78, 74, April 25, 2022 
13, Austria (National team), 79, 78, 78, April 25, 2022 
14, Czech Republic, 79, 76, 76, April 25, 2022 
15, Norway, 81, 76, 73, April 25, 2022 
16, Sweden, 79, 76, 75, April 25, 2022 
17, Ukraine, 76, 77, 74, April 25, 2022 
18, Scotland, 73, 76, 77, April 25, 2022 
19, United States, 75, 75, 74, April 25, 2022 
20, Wales, 74, 74, 72, April 25, 2022 
21, Greece, 73, 74, 73, April 25, 2022 
22, Canada, 75, 73, 70, April 25, 2022 
23, Hungary, 74, 69, 73, April 25, 2022 
24, Rep


## Fifa Ranking

&#9632; Total Points <br>
&#9632; Ranking <br>

In [6]:
URL = 'https://www.fifa.com/fifa-world-ranking/men?dateId=id13603'
webpage = requests.get(URL, headers={'Accept-Encoding': 'identity'})
soup = BeautifulSoup(webpage.text, 'lxml')
dom = etree.HTML(str(soup))
xpath = '//*[@id="content"]/main/section[2]/div/div/div[1]/table/tbody/tr[1]/td[3]/span[1]'
print(dom.xpath(xpath))


ContentDecodingError: ('Received response with content-encoding: gzip, but failed to decode it.', error('Error -3 while decompressing data: incorrect header check'))